<a href="https://colab.research.google.com/github/RAFS20/Big-Data-HDFS-Map-Reduce-Apache-Spark-Apache-Kafka/blob/main/Polars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install polars pyspark

In [31]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean, stddev
import pandas as pd
import polars as pl
import dask.dataframe as dd
import numpy as np
import time

# Configuración de Spark
spark = SparkSession.builder.appName("Comparativa").getOrCreate()

# Simulación de datos
np.random.seed(0)
data = np.random.rand(10_000_000)
pdf = pd.DataFrame({'data': data})

# DataFrame de Spark
sdf = spark.createDataFrame(pdf)

# Dask
ddf = dd.from_pandas(pdf, npartitions=10)

# Polars
pl_df = pl.DataFrame({'data': data})

# Tiempos de ejecución y estadísticas
times = {}

# Operaciones con PySpark
start_time_spark = time.time()
mean_val_spark = sdf.select(mean('data')).collect()[0][0]
median_val_spark = sdf.approxQuantile('data', [0.5], 0.01)[0]  # Aproximado
std_dev_spark = sdf.select(stddev('data')).collect()[0][0]
times['PySpark'] = time.time() - start_time_spark, mean_val_spark, median_val_spark, std_dev_spark

# Tiempo Dask
start_time_dask = time.time()
mean_val_dask = ddf['data'].mean().compute()
median_val_dask = ddf['data'].quantile(0.5).compute()
std_dev_dask = ddf['data'].std().compute()
times['Dask'] = time.time() - start_time_dask, mean_val_dask, median_val_dask, std_dev_dask

# Tiempo Polars
start_time_pl = time.time()
mean_val_pl = pl_df.select(pl.mean('data')).to_numpy()[0][0]
median_val_pl = pl_df.select(pl.median('data')).to_numpy()[0][0]
std_dev_pl = pl_df.select(pl.std('data')).to_numpy()[0][0]
times['Polars'] = time.time() - start_time_pl, mean_val_pl, median_val_pl, std_dev_pl

# Tiempo pandas
start_time_pd = time.time()
mean_val_pd = pdf['data'].mean()
median_val_pd = pdf['data'].median()
std_dev_pd = pdf['data'].std()
times['pandas'] = time.time() - start_time_pd, mean_val_pd, median_val_pd, std_dev_pd

# Tiempo NumPy
start_time_np = time.time()
mean_val_np = np.mean(data)
median_val_np = np.median(data)
std_dev_np = np.std(data)
times['NumPy'] = time.time() - start_time_np, mean_val_np, median_val_np, std_dev_np

# Calcular la herramienta más rápida
fastest_tool = min(times, key=lambda x: times[x][0])
fastest_time = times[fastest_tool][0]

# Calcular cuántas veces es más rápida la herramienta más rápida que las demás
speedups = {tool: fastest_time / time_taken[0] for tool, time_taken in times.items()}

# Imprimir resultados
print("Tiempos de ejecución y estadísticas:")
for tool, (time_taken, mean_val, median_val, std_dev) in times.items():
    print(f"{tool}: Tiempo: {time_taken:.4f} segundos, Media: {mean_val:.6f}, Mediana: {median_val:.6f}, Desv. Est.: {std_dev:.6f}")

print(f"\nLa herramienta más rápida fue {fastest_tool} con {fastest_time:.4f} segundos.")

Tiempos de ejecución y estadísticas:
PySpark: Tiempo: 20.8950 segundos, Media: 0.500000, Mediana: 0.501756, Desv. Est.: 0.288672
Dask: Tiempo: 0.2304 segundos, Media: 0.500000, Mediana: 0.500894, Desv. Est.: 0.288672
Polars: Tiempo: 0.1467 segundos, Media: 0.500000, Mediana: 0.500186, Desv. Est.: 0.288672
pandas: Tiempo: 0.2322 segundos, Media: 0.500000, Mediana: 0.500186, Desv. Est.: 0.288672
NumPy: Tiempo: 0.1328 segundos, Media: 0.500000, Mediana: 0.500186, Desv. Est.: 0.288672

La herramienta más rápida fue NumPy con 0.1328 segundos.


In [ ]:
import pandas as pd
import numpy as np
import polars as pl

# Configuración para la generación de datos
num_rows = 1000
categories = ['A', 'B', 'C', 'D']

# Generar datos simulados
np.random.seed(0)  # Para reproducibilidad
df = pd.DataFrame({
    'Category': np.random.choice(categories, num_rows),
    'Num1': np.random.rand(num_rows),
    'Num2': np.random.rand(num_rows) * 100,
    'Num3': np.random.randn(num_rows) * 50,
    'Num4': np.random.rand(num_rows) * 1000,
    'Num5': np.random.randint(0, 100, num_rows)
})

# Escribir los datos a un archivo .csv
csv_file_path = 'simulated_data.csv'
df.to_csv(csv_file_path, index=False)

# Leer el archivo .csv con Polars
df_polars = pl.read_csv(csv_file_path)

# Mostrar el DataFrame de Polars
print(df_polars)

shape: (1_000, 6)
┌──────────┬──────────┬───────────┬────────────┬────────────┬──────┐
│ Category ┆ Num1     ┆ Num2      ┆ Num3       ┆ Num4       ┆ Num5 │
│ ---      ┆ ---      ┆ ---       ┆ ---        ┆ ---        ┆ ---  │
│ str      ┆ f64      ┆ f64       ┆ f64        ┆ f64        ┆ i64  │
╞══════════╪══════════╪═══════════╪════════════╪════════════╪══════╡
│ A        ┆ 0.310381 ┆ 44.679332 ┆ -21.554817 ┆ 829.835334 ┆ 0    │
│ D        ┆ 0.373035 ┆ 83.699037 ┆ 28.637568  ┆ 941.682333 ┆ 6    │
│ B        ┆ 0.52497  ┆ 22.182403 ┆ 33.877852  ┆ 395.183827 ┆ 79   │
│ A        ┆ 0.750595 ┆ 49.394526 ┆ 65.359192  ┆ 904.131051 ┆ 21   │
│ …        ┆ …        ┆ …         ┆ …          ┆ …          ┆ …    │
│ A        ┆ 0.193623 ┆ 70.233914 ┆ 7.664423   ┆ 204.282922 ┆ 36   │
│ C        ┆ 0.11225  ┆ 72.433098 ┆ 34.706435  ┆ 569.306033 ┆ 67   │
│ B        ┆ 0.042364 ┆ 19.157101 ┆ -16.436385 ┆ 752.366306 ┆ 5    │
│ A        ┆ 0.227741 ┆ 66.217423 ┆ 33.198254  ┆ 694.951688 ┆ 60   │
└──────────┴────

In [ ]:
pip install sqlalchemy connectorx polars adbc-driver-sqlite

In [34]:
import sqlite3
import numpy as np
import pandas as pd

# Conexión (o creación si no existe) a la base de datos SQLite
conn = sqlite3.connect('test.db')

# Generar datos simulados
np.random.seed(0)  # Para reproducibilidad
num_rows = 1000
categories = ['A', 'B', 'C', 'D']  # Datos categóricos
data = {
    'Category': np.random.choice(categories, num_rows),
    'Num1': np.random.rand(num_rows),  # Distribución uniforme
    'Num2': np.random.randn(num_rows) * 10 + 50,  # Distribución normal
    'Num3': np.log(np.random.rand(num_rows) * 10 + 1),  # Distribución logarítmica
    'Num4': np.random.exponential(1, num_rows),  # Distribución exponencial
    'Num5': np.random.poisson(5, num_rows)  # Distribución de Poisson
}

df = pd.DataFrame(data)

# Crear tabla y insertar los datos en SQLite
df.to_sql('simulated_table', conn, if_exists='replace', index=False)

# Cerrar conexión a la base de datos
conn.close()

In [35]:
import polars as pl
from sqlalchemy import create_engine

# Crear el motor de conexión para SQLite
engine = create_engine('sqlite:///test.db')

# Consulta SQL para leer los datos
query = "SELECT * FROM simulated_table"

# Utilizar pl.read_database para leer los datos en un DataFrame de Polars
df_polars = pl.read_database(query=query, connection=engine.connect())

# Mostrar el DataFrame de Polars
print(df_polars)

shape: (1_000, 6)
┌──────────┬──────────┬───────────┬──────────┬──────────┬──────┐
│ Category ┆ Num1     ┆ Num2      ┆ Num3     ┆ Num4     ┆ Num5 │
│ ---      ┆ ---      ┆ ---       ┆ ---      ┆ ---      ┆ ---  │
│ str      ┆ f64      ┆ f64       ┆ f64      ┆ f64      ┆ i64  │
╞══════════╪══════════╪═══════════╪══════════╪══════════╪══════╡
│ A        ┆ 0.310381 ┆ 62.21385  ┆ 0.918714 ┆ 0.259572 ┆ 3    │
│ D        ┆ 0.373035 ┆ 48.071582 ┆ 1.937057 ┆ 1.815245 ┆ 3    │
│ B        ┆ 0.52497  ┆ 49.666807 ┆ 1.852578 ┆ 0.419246 ┆ 2    │
│ A        ┆ 0.750595 ┆ 34.691965 ┆ 1.093287 ┆ 3.87489  ┆ 5    │
│ …        ┆ …        ┆ …         ┆ …        ┆ …        ┆ …    │
│ A        ┆ 0.193623 ┆ 26.278061 ┆ 1.470373 ┆ 0.387993 ┆ 6    │
│ C        ┆ 0.11225  ┆ 36.18255  ┆ 1.445977 ┆ 1.159704 ┆ 3    │
│ B        ┆ 0.042364 ┆ 48.875562 ┆ 2.394008 ┆ 0.144141 ┆ 5    │
│ A        ┆ 0.227741 ┆ 58.978642 ┆ 2.353668 ┆ 1.635521 ┆ 2    │
└──────────┴──────────┴───────────┴──────────┴──────────┴──────┘


### Paso 1: Creación de Datos Simulados

Primero, vamos a crear un DataFrame con Polars que contenga una columna categórica (`Category`) y cinco columnas numéricas (`Num1` a `Num5`) con diferentes tipos de datos numéricos.

```python
import polars as pl
import numpy as np

# Establecer la semilla para la reproducibilidad
np.random.seed(0)

# Crear el DataFrame
df = pl.DataFrame({
    "Category": np.random.choice(["A", "B", "C", "D"], 1000),
    "Num1": np.random.rand(1000) * 100,
    "Num2": np.random.randn(1000),
    "Num3": np.random.randint(1, 100, 1000),
    "Num4": np.random.exponential(1, 1000),
    "Num5": np.random.rand(1000) * 50
})

print(df)
```



In [36]:
import polars as pl
import numpy as np

# Establecer la semilla para la reproducibilidad
np.random.seed(0)

# Crear el DataFrame
df = pl.DataFrame({
    "Category": np.random.choice(["A", "B", "C", "D"], 1000),
    "Num1": np.random.rand(1000) * 100,
    "Num2": np.random.randn(1000),
    "Num3": np.random.randint(1, 100, 1000),
    "Num4": np.random.exponential(1, 1000),
    "Num5": np.random.rand(1000) * 50
})

print(df)

shape: (1_000, 6)
┌──────────┬───────────┬───────────┬──────┬──────────┬───────────┐
│ Category ┆ Num1      ┆ Num2      ┆ Num3 ┆ Num4     ┆ Num5      │
│ ---      ┆ ---       ┆ ---       ┆ ---  ┆ ---      ┆ ---       │
│ str      ┆ f64       ┆ f64       ┆ i64  ┆ f64      ┆ f64       │
╞══════════╪═══════════╪═══════════╪══════╪══════════╪═══════════╡
│ A        ┆ 31.038083 ┆ 1.221385  ┆ 93   ┆ 0.327546 ┆ 25.425978 │
│ D        ┆ 37.303486 ┆ -0.192842 ┆ 66   ┆ 0.638946 ┆ 47.03379  │
│ B        ┆ 52.497044 ┆ -0.033319 ┆ 42   ┆ 1.280853 ┆ 43.510583 │
│ A        ┆ 75.059502 ┆ -1.530803 ┆ 39   ┆ 0.763945 ┆ 10.856792 │
│ …        ┆ …         ┆ …         ┆ …    ┆ …        ┆ …         │
│ A        ┆ 19.362329 ┆ -2.372194 ┆ 29   ┆ 1.178957 ┆ 4.428173  │
│ C        ┆ 11.224999 ┆ -1.381745 ┆ 1    ┆ 2.044436 ┆ 7.436232  │
│ B        ┆ 4.236405  ┆ -0.112444 ┆ 70   ┆ 0.26253  ┆ 7.77212   │
│ A        ┆ 22.774099 ┆ 0.897864  ┆ 58   ┆ 0.114053 ┆ 33.554484 │
└──────────┴───────────┴───────────┴──────┴─

### Ejemplo 1: Seleccionar Todas las Columnas del DataFrame

```python
all_columns = df.select(pl.all())
print(all_columns)
```

In [37]:
all_columns = df.select(pl.all())
print(all_columns)

shape: (1_000, 6)
┌──────────┬───────────┬───────────┬──────┬──────────┬───────────┐
│ Category ┆ Num1      ┆ Num2      ┆ Num3 ┆ Num4     ┆ Num5      │
│ ---      ┆ ---       ┆ ---       ┆ ---  ┆ ---      ┆ ---       │
│ str      ┆ f64       ┆ f64       ┆ i64  ┆ f64      ┆ f64       │
╞══════════╪═══════════╪═══════════╪══════╪══════════╪═══════════╡
│ A        ┆ 31.038083 ┆ 1.221385  ┆ 93   ┆ 0.327546 ┆ 25.425978 │
│ D        ┆ 37.303486 ┆ -0.192842 ┆ 66   ┆ 0.638946 ┆ 47.03379  │
│ B        ┆ 52.497044 ┆ -0.033319 ┆ 42   ┆ 1.280853 ┆ 43.510583 │
│ A        ┆ 75.059502 ┆ -1.530803 ┆ 39   ┆ 0.763945 ┆ 10.856792 │
│ …        ┆ …         ┆ …         ┆ …    ┆ …        ┆ …         │
│ A        ┆ 19.362329 ┆ -2.372194 ┆ 29   ┆ 1.178957 ┆ 4.428173  │
│ C        ┆ 11.224999 ┆ -1.381745 ┆ 1    ┆ 2.044436 ┆ 7.436232  │
│ B        ┆ 4.236405  ┆ -0.112444 ┆ 70   ┆ 0.26253  ┆ 7.77212   │
│ A        ┆ 22.774099 ┆ 0.897864  ┆ 58   ┆ 0.114053 ┆ 33.554484 │
└──────────┴───────────┴───────────┴──────┴─

### Ejemplo 2: Seleccionar Columnas Específicas por Nombre

```python
specific_columns = df.select(["Category", "Num1", "Num3"])
print(specific_columns)
```

In [38]:
specific_columns = df.select(["Category", "Num1", "Num3"])
print(specific_columns)

shape: (1_000, 3)
┌──────────┬───────────┬──────┐
│ Category ┆ Num1      ┆ Num3 │
│ ---      ┆ ---       ┆ ---  │
│ str      ┆ f64       ┆ i64  │
╞══════════╪═══════════╪══════╡
│ A        ┆ 31.038083 ┆ 93   │
│ D        ┆ 37.303486 ┆ 66   │
│ B        ┆ 52.497044 ┆ 42   │
│ A        ┆ 75.059502 ┆ 39   │
│ …        ┆ …         ┆ …    │
│ A        ┆ 19.362329 ┆ 29   │
│ C        ┆ 11.224999 ┆ 1    │
│ B        ┆ 4.236405  ┆ 70   │
│ A        ┆ 22.774099 ┆ 58   │
└──────────┴───────────┴──────┘


### Ejemplo 3: Seleccionar Columnas


```python
# En este ejemplo específico, sabemos que 'Num1' a 'Num5' son las columnas numéricas
numeric_columns = df.select([pl.col("Num1"), pl.col("Num2"), pl.col("Num3"), pl.col("Num4"), pl.col("Num5")])
print(numeric_columns)
```

In [39]:
# En este ejemplo específico, sabemos que 'Num1' a 'Num5' son las columnas numéricas
numeric_columns = df.select([pl.col("Num1"), pl.col("Num2"), pl.col("Num3"), pl.col("Num4"), pl.col("Num5")])
print(numeric_columns)

shape: (1_000, 5)
┌───────────┬───────────┬──────┬──────────┬───────────┐
│ Num1      ┆ Num2      ┆ Num3 ┆ Num4     ┆ Num5      │
│ ---       ┆ ---       ┆ ---  ┆ ---      ┆ ---       │
│ f64       ┆ f64       ┆ i64  ┆ f64      ┆ f64       │
╞═══════════╪═══════════╪══════╪══════════╪═══════════╡
│ 31.038083 ┆ 1.221385  ┆ 93   ┆ 0.327546 ┆ 25.425978 │
│ 37.303486 ┆ -0.192842 ┆ 66   ┆ 0.638946 ┆ 47.03379  │
│ 52.497044 ┆ -0.033319 ┆ 42   ┆ 1.280853 ┆ 43.510583 │
│ 75.059502 ┆ -1.530803 ┆ 39   ┆ 0.763945 ┆ 10.856792 │
│ …         ┆ …         ┆ …    ┆ …        ┆ …         │
│ 19.362329 ┆ -2.372194 ┆ 29   ┆ 1.178957 ┆ 4.428173  │
│ 11.224999 ┆ -1.381745 ┆ 1    ┆ 2.044436 ┆ 7.436232  │
│ 4.236405  ┆ -0.112444 ┆ 70   ┆ 0.26253  ┆ 7.77212   │
│ 22.774099 ┆ 0.897864  ┆ 58   ┆ 0.114053 ┆ 33.554484 │
└───────────┴───────────┴──────┴──────────┴───────────┘



### FILTROS

Paso Adicional: Agregar Columna de Fechas



In [51]:
import polars as pl
import numpy as np

# Generar fechas aleatorias (corrección para crear un rango de fechas)
start_date = np.datetime64('2020-01-01')
end_date = np.datetime64('2020-12-31')
dates = np.arange(start_date, end_date, dtype='datetime64[D]')
random_dates = np.random.choice(dates, 1000)

# Crear DataFrame de Polars
df = pl.DataFrame({
    "Category": np.random.choice(["A", "B", "C", "D"], 1000),
    "Num1": np.random.rand(1000) * 100,
    "Num2": np.random.randn(1000),
    "Num3": np.random.randint(1, 100, 1000),
    "Num4": np.random.exponential(1, 1000),
    "Num5": np.random.rand(1000) * 50,
    "Date": random_dates
})

# Asegurarse de que la columna 'Date' es del tipo fecha
df = df.with_columns(df["Date"].cast(pl.Date))


### Ejemplo 1: Filtrar Filas Basadas en una Condición Simple

Vamos a filtrar las filas donde el valor de `Num1` sea mayor a 50.

```python
filtered_simple = df.filter(pl.col("Num1") > 50)
print(filtered_simple)
```



In [52]:
filtered_simple = df.filter(pl.col("Num1") > 50)
print(filtered_simple)

shape: (497, 7)
┌──────────┬───────────┬───────────┬──────┬──────────┬───────────┬────────────┐
│ Category ┆ Num1      ┆ Num2      ┆ Num3 ┆ Num4     ┆ Num5      ┆ Date       │
│ ---      ┆ ---       ┆ ---       ┆ ---  ┆ ---      ┆ ---       ┆ ---        │
│ str      ┆ f64       ┆ f64       ┆ i64  ┆ f64      ┆ f64       ┆ date       │
╞══════════╪═══════════╪═══════════╪══════╪══════════╪═══════════╪════════════╡
│ C        ┆ 51.122014 ┆ -1.404708 ┆ 56   ┆ 0.542551 ┆ 11.533433 ┆ 2020-03-25 │
│ D        ┆ 77.125256 ┆ -0.36575  ┆ 95   ┆ 0.540152 ┆ 12.3826   ┆ 2020-02-19 │
│ D        ┆ 59.686489 ┆ 0.275136  ┆ 1    ┆ 0.946497 ┆ 11.835743 ┆ 2020-05-04 │
│ A        ┆ 54.643893 ┆ -1.558889 ┆ 4    ┆ 0.305849 ┆ 32.209998 ┆ 2020-09-26 │
│ …        ┆ …         ┆ …         ┆ …    ┆ …        ┆ …         ┆ …          │
│ A        ┆ 94.20893  ┆ 0.781847  ┆ 25   ┆ 0.075035 ┆ 7.74581   ┆ 2020-05-15 │
│ A        ┆ 62.915617 ┆ 1.305445  ┆ 51   ┆ 0.544497 ┆ 6.520941  ┆ 2020-02-25 │
│ D        ┆ 51.554813 ┆

### Ejemplo 2: Filtrar Utilizando Condiciones Compuestas

Ahora, filtraremos las filas que cumplan con dos condiciones: `Num1` mayor a 50 y `Num3` menor que 30.

```python
filtered_compound = df.filter((pl.col("Num1") > 50) & (pl.col("Num3") < 30))
print(filtered_compound)
```

In [53]:
filtered_compound = df.filter((pl.col("Num1") > 50) & (pl.col("Num3") < 30))
print(filtered_compound)

shape: (143, 7)
┌──────────┬───────────┬───────────┬──────┬──────────┬───────────┬────────────┐
│ Category ┆ Num1      ┆ Num2      ┆ Num3 ┆ Num4     ┆ Num5      ┆ Date       │
│ ---      ┆ ---       ┆ ---       ┆ ---  ┆ ---      ┆ ---       ┆ ---        │
│ str      ┆ f64       ┆ f64       ┆ i64  ┆ f64      ┆ f64       ┆ date       │
╞══════════╪═══════════╪═══════════╪══════╪══════════╪═══════════╪════════════╡
│ D        ┆ 59.686489 ┆ 0.275136  ┆ 1    ┆ 0.946497 ┆ 11.835743 ┆ 2020-05-04 │
│ A        ┆ 54.643893 ┆ -1.558889 ┆ 4    ┆ 0.305849 ┆ 32.209998 ┆ 2020-09-26 │
│ A        ┆ 60.433745 ┆ -0.985396 ┆ 25   ┆ 2.825535 ┆ 30.406909 ┆ 2020-11-03 │
│ D        ┆ 50.434605 ┆ -1.213214 ┆ 19   ┆ 1.707626 ┆ 5.26124   ┆ 2020-12-11 │
│ …        ┆ …         ┆ …         ┆ …    ┆ …        ┆ …         ┆ …          │
│ A        ┆ 51.34045  ┆ 0.123047  ┆ 17   ┆ 2.863434 ┆ 27.997369 ┆ 2020-03-21 │
│ B        ┆ 75.806398 ┆ 0.49938   ┆ 22   ┆ 0.222167 ┆ 27.550116 ┆ 2020-09-07 │
│ B        ┆ 63.669541 ┆

### Ejemplo 3: Filtrar Fechas Dentro de un Rango Específico



In [56]:
from datetime import datetime
import polars as pl

filtered_range_df = df.filter(
    pl.col("Date").is_between(pl.lit(datetime(2020, 3, 1)), pl.lit(datetime(2020, 6, 30)))
)

print(filtered_range_df)


shape: (337, 7)
┌──────────┬───────────┬───────────┬──────┬──────────┬───────────┬────────────┐
│ Category ┆ Num1      ┆ Num2      ┆ Num3 ┆ Num4     ┆ Num5      ┆ Date       │
│ ---      ┆ ---       ┆ ---       ┆ ---  ┆ ---      ┆ ---       ┆ ---        │
│ str      ┆ f64       ┆ f64       ┆ i64  ┆ f64      ┆ f64       ┆ date       │
╞══════════╪═══════════╪═══════════╪══════╪══════════╪═══════════╪════════════╡
│ C        ┆ 51.122014 ┆ -1.404708 ┆ 56   ┆ 0.542551 ┆ 11.533433 ┆ 2020-03-25 │
│ D        ┆ 59.686489 ┆ 0.275136  ┆ 1    ┆ 0.946497 ┆ 11.835743 ┆ 2020-05-04 │
│ D        ┆ 57.760586 ┆ -0.368788 ┆ 63   ┆ 0.385142 ┆ 35.672454 ┆ 2020-04-22 │
│ C        ┆ 16.703196 ┆ 0.327954  ┆ 29   ┆ 0.307399 ┆ 47.182164 ┆ 2020-05-20 │
│ …        ┆ …         ┆ …         ┆ …    ┆ …        ┆ …         ┆ …          │
│ B        ┆ 93.2006   ┆ -0.611262 ┆ 71   ┆ 1.803935 ┆ 37.72623  ┆ 2020-06-15 │
│ B        ┆ 63.669541 ┆ 0.227978  ┆ 10   ┆ 0.446578 ┆ 49.916981 ┆ 2020-06-20 │
│ A        ┆ 94.20893  ┆

Ejemplo: Añadir una Nueva Columna Calculada a Partir de Otras Columnas

Vamos a añadir una nueva columna llamada "Num6", que será el resultado de multiplicar "Num1" por "Num2".

In [58]:
df_with_new_column = df.with_columns((pl.col("Num1") * pl.col("Num2")).alias("Num6"))
print(df_with_new_column)

shape: (1_000, 8)
┌──────────┬───────────┬───────────┬──────┬──────────┬───────────┬────────────┬─────────────┐
│ Category ┆ Num1      ┆ Num2      ┆ Num3 ┆ Num4     ┆ Num5      ┆ Date       ┆ Num6        │
│ ---      ┆ ---       ┆ ---       ┆ ---  ┆ ---      ┆ ---       ┆ ---        ┆ ---         │
│ str      ┆ f64       ┆ f64       ┆ i64  ┆ f64      ┆ f64       ┆ date       ┆ f64         │
╞══════════╪═══════════╪═══════════╪══════╪══════════╪═══════════╪════════════╪═════════════╡
│ B        ┆ 15.380319 ┆ -0.305241 ┆ 27   ┆ 1.023416 ┆ 17.620948 ┆ 2020-07-18 ┆ -4.694705   │
│ C        ┆ 51.122014 ┆ -1.404708 ┆ 56   ┆ 0.542551 ┆ 11.533433 ┆ 2020-03-25 ┆ -71.811502  │
│ D        ┆ 77.125256 ┆ -0.36575  ┆ 95   ┆ 0.540152 ┆ 12.3826   ┆ 2020-02-19 ┆ -28.20858   │
│ C        ┆ 41.305362 ┆ -0.097275 ┆ 83   ┆ 2.09784  ┆ 4.551589  ┆ 2020-01-12 ┆ -4.017961   │
│ …        ┆ …         ┆ …         ┆ …    ┆ …        ┆ …         ┆ …          ┆ …           │
│ C        ┆ 23.576566 ┆ 0.684312  ┆ 73   

Ejemplo: Modificar Columnas Existentes Aplicando Transformaciones

Vamos a normalizar la columna "Num3" (ponerla en una escala de 0 a 1) utilizando el máximo y el mínimo de esa columna.

In [59]:
df_modified = df.with_columns(
    ((pl.col("Num3") - pl.col("Num3").min()) / (pl.col("Num3").max() - pl.col("Num3").min())).alias("Num3_normalized")
)
print(df_modified)

shape: (1_000, 8)
┌──────────┬───────────┬───────────┬──────┬──────────┬───────────┬────────────┬─────────────────┐
│ Category ┆ Num1      ┆ Num2      ┆ Num3 ┆ Num4     ┆ Num5      ┆ Date       ┆ Num3_normalized │
│ ---      ┆ ---       ┆ ---       ┆ ---  ┆ ---      ┆ ---       ┆ ---        ┆ ---             │
│ str      ┆ f64       ┆ f64       ┆ i64  ┆ f64      ┆ f64       ┆ date       ┆ f64             │
╞══════════╪═══════════╪═══════════╪══════╪══════════╪═══════════╪════════════╪═════════════════╡
│ B        ┆ 15.380319 ┆ -0.305241 ┆ 27   ┆ 1.023416 ┆ 17.620948 ┆ 2020-07-18 ┆ 0.265306        │
│ C        ┆ 51.122014 ┆ -1.404708 ┆ 56   ┆ 0.542551 ┆ 11.533433 ┆ 2020-03-25 ┆ 0.561224        │
│ D        ┆ 77.125256 ┆ -0.36575  ┆ 95   ┆ 0.540152 ┆ 12.3826   ┆ 2020-02-19 ┆ 0.959184        │
│ C        ┆ 41.305362 ┆ -0.097275 ┆ 83   ┆ 2.09784  ┆ 4.551589  ┆ 2020-01-12 ┆ 0.836735        │
│ …        ┆ …         ┆ …         ┆ …    ┆ …        ┆ …         ┆ …          ┆ …               │
│ 

Ejemplo: Crear Columnas Indicadoras o Flags Basadas en Condiciones Específicas

Añadiremos una columna indicadora llamada "High_Num1" que será True si el valor de "Num1" es mayor que 50, y False en caso contrario.

In [61]:
df_with_flag = df.with_columns(
    (pl.col("Num1") > 50).alias("High_Num1")
)
print(df_with_flag)

shape: (1_000, 8)
┌──────────┬───────────┬───────────┬──────┬──────────┬───────────┬────────────┬───────────┐
│ Category ┆ Num1      ┆ Num2      ┆ Num3 ┆ Num4     ┆ Num5      ┆ Date       ┆ High_Num1 │
│ ---      ┆ ---       ┆ ---       ┆ ---  ┆ ---      ┆ ---       ┆ ---        ┆ ---       │
│ str      ┆ f64       ┆ f64       ┆ i64  ┆ f64      ┆ f64       ┆ date       ┆ bool      │
╞══════════╪═══════════╪═══════════╪══════╪══════════╪═══════════╪════════════╪═══════════╡
│ B        ┆ 15.380319 ┆ -0.305241 ┆ 27   ┆ 1.023416 ┆ 17.620948 ┆ 2020-07-18 ┆ false     │
│ C        ┆ 51.122014 ┆ -1.404708 ┆ 56   ┆ 0.542551 ┆ 11.533433 ┆ 2020-03-25 ┆ true      │
│ D        ┆ 77.125256 ┆ -0.36575  ┆ 95   ┆ 0.540152 ┆ 12.3826   ┆ 2020-02-19 ┆ true      │
│ C        ┆ 41.305362 ┆ -0.097275 ┆ 83   ┆ 2.09784  ┆ 4.551589  ┆ 2020-01-12 ┆ false     │
│ …        ┆ …         ┆ …         ┆ …    ┆ …        ┆ …         ┆ …          ┆ …         │
│ C        ┆ 23.576566 ┆ 0.684312  ┆ 73   ┆ 0.398554 ┆ 38.6428

Ejemplo: Agrupar Datos por una o Varias Columnas y Calcular Estadísticas Resumidas

Agruparemos por la columna "Category" y calcularemos la suma, la media y el conteo para la columna "Num1".

In [62]:
grouped_summary = df.groupby("Category").agg([
    pl.col("Num1").sum().alias("Sum_Num1"),
    pl.col("Num1").mean().alias("Mean_Num1"),
    pl.count("Num1").alias("Count_Num1")
])
print(grouped_summary)

shape: (4, 4)
┌──────────┬──────────────┬───────────┬────────────┐
│ Category ┆ Sum_Num1     ┆ Mean_Num1 ┆ Count_Num1 │
│ ---      ┆ ---          ┆ ---       ┆ ---        │
│ str      ┆ f64          ┆ f64       ┆ u32        │
╞══════════╪══════════════╪═══════════╪════════════╡
│ A        ┆ 11837.4729   ┆ 49.737281 ┆ 238        │
│ C        ┆ 14099.390982 ┆ 50.535452 ┆ 279        │
│ B        ┆ 11212.227697 ┆ 46.717615 ┆ 240        │
│ D        ┆ 12532.298765 ┆ 51.573246 ┆ 243        │
└──────────┴──────────────┴───────────┴────────────┘


<ipython-input-62-3427ddf6c96a>:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  grouped_summary = df.groupby("Category").agg([


Ejemplo: Agrupar y Aplicar Múltiples Agregaciones en una Sola Pasada

Agruparemos por la columna "Category" y aplicaremos múltiples agregaciones (suma y media) a varias columnas ("Num1", "Num2").

In [63]:
multiple_aggregations = df.groupby("Category").agg([
    pl.col("Num1").sum().alias("Sum_Num1"),
    pl.col("Num1").mean().alias("Mean_Num1"),
    pl.col("Num2").sum().alias("Sum_Num2"),
    pl.col("Num2").mean().alias("Mean_Num2")
])
print(multiple_aggregations)

shape: (4, 5)
┌──────────┬──────────────┬───────────┬────────────┬───────────┐
│ Category ┆ Sum_Num1     ┆ Mean_Num1 ┆ Sum_Num2   ┆ Mean_Num2 │
│ ---      ┆ ---          ┆ ---       ┆ ---        ┆ ---       │
│ str      ┆ f64          ┆ f64       ┆ f64        ┆ f64       │
╞══════════╪══════════════╪═══════════╪════════════╪═══════════╡
│ B        ┆ 11212.227697 ┆ 46.717615 ┆ -2.320906  ┆ -0.00967  │
│ C        ┆ 14099.390982 ┆ 50.535452 ┆ -13.600111 ┆ -0.048746 │
│ D        ┆ 12532.298765 ┆ 51.573246 ┆ -12.242603 ┆ -0.050381 │
│ A        ┆ 11837.4729   ┆ 49.737281 ┆ -0.171865  ┆ -0.000722 │
└──────────┴──────────────┴───────────┴────────────┴───────────┘


<ipython-input-63-fcc39d6e0486>:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  multiple_aggregations = df.groupby("Category").agg([


Ejemplo: Realizar Agregaciones Condicionales Basadas en Valores de Otra Columna

Para este ejemplo, agruparemos por "Category" y calcularemos la suma de "Num1" solo para aquellos valores de "Num2" que sean mayores que 0.

In [64]:
conditional_aggregation = df.groupby("Category").agg([
    pl.when(pl.col("Num2") > 0).then(pl.col("Num1")).sum().alias("Conditional_Sum_Num1")
])
print(conditional_aggregation)

shape: (4, 2)
┌──────────┬──────────────────────┐
│ Category ┆ Conditional_Sum_Num1 │
│ ---      ┆ ---                  │
│ str      ┆ f64                  │
╞══════════╪══════════════════════╡
│ A        ┆ 6242.098635          │
│ B        ┆ 4905.767514          │
│ C        ┆ 6933.433809          │
│ D        ┆ 5971.785662          │
└──────────┴──────────────────────┘


<ipython-input-64-3d213d6d07bb>:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  conditional_aggregation = df.groupby("Category").agg([


Ejemplo: Combinar Selección, Filtrado y Agregación

Imagina que queremos calcular la media de la columna "Num1" para los registros donde "Num3" sea mayor que 50, y además, solo para la categoría "B" en la columna "Category".

In [65]:
# Primero, filtramos por la condición específica, luego seleccionamos la columna de interés, y finalmente calculamos la media.
mean_num1_for_category_B_and_num3_gt_50 = df.filter(
    (pl.col("Category") == "B") & (pl.col("Num3") > 50)
).select(
    pl.mean("Num1")
)

print(mean_num1_for_category_B_and_num3_gt_50)

shape: (1, 1)
┌───────────┐
│ Num1      │
│ ---       │
│ f64       │
╞═══════════╡
│ 48.497906 │
└───────────┘


Ejemplo: Uso de Expresiones para Crear Consultas Encadenadas Eficientes

Supongamos que queremos agregar una columna que indique si "Num2" está por encima de su media global y luego calcular la suma de "Num1" solo para esos casos, todo en una sola pasada.

In [67]:
# Añadimos una columna indicadora para Num2 > mean(Num2), luego filtramos por esa condición y calculamos la suma de Num1.
sum_num1_where_num2_above_mean = df.with_columns(
    (pl.col("Num2") > pl.mean("Num2")).alias("Num2_above_mean")
).filter(
    pl.col("Num2_above_mean")
).select(
    pl.sum("Num1")
)

print(sum_num1_where_num2_above_mean)

shape: (1, 1)
┌──────────────┐
│ Num1         │
│ ---          │
│ f64          │
╞══════════════╡
│ 24757.667843 │
└──────────────┘


# JOINS

In [68]:
import polars as pl

# DataFrame 1
df1 = pl.DataFrame({
    "id": [1, 2, 3, 4],
    "value_df1": ["A", "B", "C", "D"]
})

# DataFrame 2
df2 = pl.DataFrame({
    "id": [3, 4, 5, 6],
    "value_df2": ["E", "F", "G", "H"]
})

Ejemplo: Inner Join

El inner join devuelve las filas que tienen claves coincidentes en ambos DataFrames.

In [69]:
inner_joined_df = df1.join(df2, on="id", how="inner")
print(inner_joined_df)

shape: (2, 3)
┌─────┬───────────┬───────────┐
│ id  ┆ value_df1 ┆ value_df2 │
│ --- ┆ ---       ┆ ---       │
│ i64 ┆ str       ┆ str       │
╞═════╪═══════════╪═══════════╡
│ 3   ┆ C         ┆ E         │
│ 4   ┆ D         ┆ F         │
└─────┴───────────┴───────────┘


Ejemplo: Left Join

El left join devuelve todas las filas del DataFrame izquierdo y las filas coincidentes del DataFrame derecho. Las filas del DataFrame izquierdo que no tienen coincidencias en el DataFrame derecho tienen valores null en las columnas del DataFrame derecho.

In [70]:
left_joined_df = df1.join(df2, on="id", how="left")
print(left_joined_df)

shape: (4, 3)
┌─────┬───────────┬───────────┐
│ id  ┆ value_df1 ┆ value_df2 │
│ --- ┆ ---       ┆ ---       │
│ i64 ┆ str       ┆ str       │
╞═════╪═══════════╪═══════════╡
│ 1   ┆ A         ┆ null      │
│ 2   ┆ B         ┆ null      │
│ 3   ┆ C         ┆ E         │
│ 4   ┆ D         ┆ F         │
└─────┴───────────┴───────────┘


Simulación de Right Join

Dado que Polars no tiene un "right join" directo, invertimos el orden de los DataFrames y usamos un "left join".

In [73]:
# Simulación de Right Join invirtiendo el orden y usando un left join
right_joined_df = df2.join(df1, on="id", how="left")
print(right_joined_df)

shape: (4, 3)
┌─────┬───────────┬───────────┐
│ id  ┆ value_df2 ┆ value_df1 │
│ --- ┆ ---       ┆ ---       │
│ i64 ┆ str       ┆ str       │
╞═════╪═══════════╪═══════════╡
│ 3   ┆ E         ┆ C         │
│ 4   ┆ F         ┆ D         │
│ 5   ┆ G         ┆ null      │
│ 6   ┆ H         ┆ null      │
└─────┴───────────┴───────────┘


Ejemplo: Outer Join

El outer join devuelve todas las filas de ambos DataFrames, con filas coincidentes de ambos lados donde estén disponibles. Si no hay coincidencia, el lado correspondiente tendrá valores null.

In [74]:
outer_joined_df = df1.join(df2, on="id", how="outer")
print(outer_joined_df)

shape: (6, 4)
┌──────┬───────────┬──────────┬───────────┐
│ id   ┆ value_df1 ┆ id_right ┆ value_df2 │
│ ---  ┆ ---       ┆ ---      ┆ ---       │
│ i64  ┆ str       ┆ i64      ┆ str       │
╞══════╪═══════════╪══════════╪═══════════╡
│ 3    ┆ C         ┆ 3        ┆ E         │
│ 4    ┆ D         ┆ 4        ┆ F         │
│ null ┆ null      ┆ 5        ┆ G         │
│ null ┆ null      ┆ 6        ┆ H         │
│ 2    ┆ B         ┆ null     ┆ null      │
│ 1    ┆ A         ┆ null     ┆ null      │
└──────┴───────────┴──────────┴───────────┘


Ejemplo: Utilizar Joins para Enriquecer un DataFrame con Información de Otro

Supongamos que queremos enriquecer df1 con la información de df2 basándonos en la columna "id". Podemos usar un left join para asegurarnos de conservar todas las filas de df1 mientras traemos los datos correspondientes de df2.

In [72]:
enriched_df = df1.join(df2, on="id", how="left")
print(enriched_df)

shape: (4, 3)
┌─────┬───────────┬───────────┐
│ id  ┆ value_df1 ┆ value_df2 │
│ --- ┆ ---       ┆ ---       │
│ i64 ┆ str       ┆ str       │
╞═════╪═══════════╪═══════════╡
│ 1   ┆ A         ┆ null      │
│ 2   ┆ B         ┆ null      │
│ 3   ┆ C         ┆ E         │
│ 4   ┆ D         ┆ F         │
└─────┴───────────┴───────────┘
